[[source]](../api/alibi.explainers/similarity/grad.rst)

# Similarity explanations

## Overview

## Usage

#### Initialization

```python
from alibi.explainers.similarity.grad import GradientSimilarity

explainer = GradientSimilarity(model, 
                               loss_fn)
```


#### Fit

Fitting is straightforward, just passing the training set:
```python
explainer.fit(X_train, y_train)
```

#### Explanation

We can now explain the instance:

```python
explanation = explainer.explain(X, y)
```

where:

* `X` - test instances to be explained.

* `y` - target class. This array can contain either a single entrance that is applied for all test instances or multiple entrances, one for each test instance.

## Examples

[Similarity explanation on MNIST](../examples/similarity_explanations_mnist.ipynb)
